In [ ]:
import imageio.v3
import base64
import os
import requests
from typing import List
from fastapi import HTTPException


In [ ]:
def video_to_base64_array(video_path: str) -> List[str]:
    frames = []

    reader = imageio.get_reader(video_path, 'ffmpeg')
    for frame in reader:
        buffer = imageio.imwrite(imageio.RETURN_BYTES, frame, format='JPEG')
        base64_frame = base64.b64encode(buffer)
        frames.append(base64_frame.decode('utf-8'))
    reader.close()

    return frames

In [ ]:
frames = video_to_base64_array('files/video.mp4')

In [ ]:
def base64_array_to_video(frames: List[str], output_path: str):
    writer = imageio.get_writer(output_path, fps=24)
    for frame in frames:
        buffer = base64.b64decode(frame)
        writer.append_data(imageio.imread(buffer, format='JPEG'))
    writer.close()



In [ ]:
def remove_video_background(frames: List[str], model: str = 'u2net') -> List[str]:
    # Parâmetros da solicitação para a API rembg
    url = "http://127.0.0.1:7860/rembg"
    processed_frames = []
    for frame in frames:
        data = {
        "input_image": frame,
        "model": model,
        "return_mask": False,
        "alpha_matting": False,
        "alpha_matting_foreground_threshold": 240,
        "alpha_matting_background_threshold": 10,
        "alpha_matting_erode_size": 10
        }
         # Enviar a solicitação para a API rembg
        response = requests.post(url, json=data)
        print(response.json())
    
        # Verificar a resposta
        if response.status_code == 200:
            responde_result =  response
            #criar uma lista para armazenar as imagens processadas em base64
            processed_frames.append(responde_result.json()["image"])
        else:
            raise HTTPException(status_code=500, detail="Internal server error")
    return processed_frames


           
           

        


In [ ]:
frames

In [ ]:
processed_frames_bse64 = remove_video_background(frames)

In [ ]:
processed_frames_bse64

In [ ]:
base64_array_to_video(processed_frames_bse64, 'files/video_output.mp4')

In [ ]:
import sys 
a = sys.getsizeof(processed_frames_bse64)
print(a)